In [ ]:
# default_exp main

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Loading

> Module to load the roster table.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
from pathlib import Path
import re
import pandas as pd
from typing import Any
from datetime import timedelta
from roster2ical.roster import Shift, ShiftProperties, Roster

In [ ]:
data_path = Path.cwd() / "data"
roster_path = data_path / "202201_example_roster.xlsx"
# roster_path = data_path / "private" / "202201_Jan.xlsx"

day_pattern = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
date_pattern = re.compile(r"\d{2}")


def daycol(col):
    return day_pattern.search(col)


# TODO: Should use daycols for variable length months
df = pd.read_excel(roster_path, skiprows=0, header=1, index_col=0)
roster_df = (
    df.dropna().rename_axis("Name").loc[:, [col for col in df.columns if daycol(col)]]
)
roster_df

,SA 01,SO 02,MO 03,DI 04,MI 05,DO 06,FR 07,SA 08,SO 09,MO 10,...,SA 22,SO 23,MO 24,DI 25,MI 26,DO 27,FR 28,SA 29,SO 30,MO 31
Name,,,,,,,,,,,,,,,,,,,,,
"Müller, Hans",(/),(/),F0,F0,F0,(/),F0,(/),(/),F0,...,(/),(/),F0,F0,F0,F0,F0,(/),(/),F0
"Vogel, Jürgen",(/),(/),F0,F0,F0,(/),F0,(/),(/),F0,...,(/),(/),F0,F0,F0,F0,F0,(/),(/),F0
"Clooney, Georg",(/),(/),F1,F1,F1,(/),N1,N2,N2,(/),...,U,U,N1,N1,(/),(/),F1,F2,F2,S1
"Springer, Axel",(/),(/),(/),S1,(/),(/),S1,(/),(/),(/),...,U,U,U,U,U,U,U,U,U,N1
"Schiffer, Claudia",(/),N2,N1,(/),(/),(/),U,(/),(/),S1,...,(/),(/),S1,ZA,ZA,S1,N1,N2,N2,(/)
"Twist, Oliver",(/),(/),U,U,U,(/),3g,(/),(/),N1,...,(/),(/),3g,3g,3g,3g,3g,(/),(/),3g
"Kerner, Johannes B.",(/),(/),S1,N1,N1,N2,(/),(/),(/),(/),...,F2,F2,(/),S1,N1,N1,(/),(/),(/),F1
"Hagen, Nina",F2,F2,(/),(/),S1,(/),(/),(/),(/),(/),...,(/),(/),ZA,(/),(/),(/),S1,(/),(/),ZA
"Klum, Heidi",N2,(/),(/),(/),(/),(/),F1,F2,F2,(/),...,N2,N2,(/),(/),F1,F1,F1,U,U,U


In [ ]:
abbr2shiftproperties = {
    "(/)": None,
    "2M": None,
    "3g": None,
    "F0": ShiftProperties(
        name="Früh0", starting_hour=timedelta(hours=8), duration=timedelta(hours=10)
    ),
    "F1": ShiftProperties(
        name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "F2": ShiftProperties(
        name="Früh2", starting_hour=timedelta(hours=8), duration=timedelta(hours=12)
    ),
    "FB": ShiftProperties(
        name="Fortbildung",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
    "N1": ShiftProperties(
        name="Nacht1", starting_hour=timedelta(hours=22), duration=timedelta(hours=10)
    ),
    "N2": ShiftProperties(
        name="Nacht2", starting_hour=timedelta(hours=20), duration=timedelta(hours=12)
    ),
    "S1": ShiftProperties(
        name="Spät1", starting_hour=timedelta(hours=15), duration=timedelta(hours=8)
    ),
    "U": ShiftProperties(
        name="Urlaub", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "ZA": ShiftProperties(
        name="Zeitausgleich",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
}


In [ ]:
r = Roster.from_dict(
    {
        "Mo 01": "F1",
        "Di 02": "N1",
    },
    mapper=_abbr2shiftproperties,
)
with (data_path / "output" / "example.ics").open("w") as f:
    f.write(str(r.to_ics()))
c = r.to_ics()
c.events

{<Event 'Früh1' begin:2022-01-01T08:00:00+00:00 end:2022-01-01T16:00:00+00:00>,
 <Event 'Nacht1' begin:2022-01-02T22:00:00+00:00 end:2022-01-03T08:00:00+00:00>}

In [ ]:
roster_df.loc["Clooney, Georg"].to_dict()

{'SA 01': '(/)',
 'SO 02': '(/)',
 'MO 03': 'F1',
 'DI 04': 'F1',
 'MI 05': 'F1',
 'DO 06': '(/)',
 'FR 07': 'N1',
 'SA 08': 'N2',
 'SO 09': 'N2',
 'MO 10': '(/)',
 'DI 11': '(/)',
 'MI 12': 'S1',
 'DO 13': '(/)',
 'FR 14': '(/)',
 'SA 15': 'U',
 'SO 16': 'U',
 'MO 17': 'U',
 'DI 18': 'U',
 'MI 19': 'U',
 'DO 20': 'U',
 'FR 21': 'U',
 'SA 22': 'U',
 'SO 23': 'U',
 'MO 24': 'N1',
 'DI 25': 'N1',
 'MI 26': '(/)',
 'DO 27': '(/)',
 'FR 28': 'F1',
 'SA 29': 'F2',
 'SO 30': 'F2',
 'MO 31': 'S1'}

In [ ]:
name = "Clooney, Georg"
r = Roster.from_dict(roster_df.loc[name].to_dict(), mapper=_abbr2shiftproperties)
f1: Shift = r.shifts[0]
f1.beginning

datetime.datetime(2022, 1, 3, 8, 0)

In [ ]:
str(r.to_ics())

'BEGIN:VCALENDAR\r\nVERSION:2.0\r\nPRODID:ics.py - http://git.io/lLljaA\r\nBEGIN:VEVENT\r\nDURATION:PT8H\r\nDTSTART:20220119T080000Z\r\nSUMMARY:Urlaub\r\nUID:8dc1946c-2d3d-4c3d-86d2-3a360cf8e0e9@8dc1.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT10H\r\nDTSTART:20220125T220000Z\r\nSUMMARY:Nacht1\r\nUID:e51636ad-6cf4-469e-91c3-7684c6268d1f@e516.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT8H\r\nDTSTART:20220128T080000Z\r\nSUMMARY:Früh1\r\nUID:c7f762c4-5fd3-4aaa-a7c4-b73c5e27c59c@c7f7.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT8H\r\nDTSTART:20220131T150000Z\r\nSUMMARY:Spät1\r\nUID:1d9006e6-831e-47ce-a151-8367474cd2a0@1d90.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT12H\r\nDTSTART:20220130T080000Z\r\nSUMMARY:Früh2\r\nUID:c9fdb843-15d9-4607-a588-f40a13ba2850@c9fd.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT10H\r\nDTSTART:20220124T220000Z\r\nSUMMARY:Nacht1\r\nUID:c6a5a7a6-e655-4014-ab91-a672d25181b3@c6a5.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDURATION:PT8H\r\nDTSTART:20220112

In [ ]:
for name in roster_df.index:
    r = Roster.from_dict(roster_df.loc[name].to_dict(), mapper=_abbr2shiftproperties)
    filename = f"{name.replace(',', '_').replace(' ', '')}.ics"
    with (data_path / "output" / filename).open("w") as f:
        f.write(str(r.to_ics()))
